# Sistemas de recomendacion

## 1. Sistema basado en el usuario

Dataset

In [1]:
import numpy as np
n_users = 500
users = np.zeros(shape=(n_users,5))

for i in range(n_users):
  users[i,0] = np.random.randint(0,10)
  users[i,1] = np.random.randint(0,10)
  users[i,2] = np.random.randint(0,4)
  users[i,3] = np.random.randint(0,5)
  users[i,4] = np.random.randn()

In [2]:
users.shape

(500, 5)

In [3]:
n_items = 25
user_items = np.random.randint(0, n_items, size=(n_users, 4))

In [4]:
user_items.shape

(500, 4)

Algoritmo de sistemas de recomendacion

In [5]:
# Agrupamos a los usuarios
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=12, radius=1.8)

In [6]:
knn.fit(users)

NearestNeighbors(n_neighbors=12, radius=1.8)

In [7]:
# Nuevo usuario
test_user = np.array([8,2,1,2,0.79])
dist, kusers = knn.kneighbors(test_user.reshape(-1,5))

In [8]:
kusers

array([[ 11, 391, 175, 413, 430, 420, 246, 270,  30,  97, 167, 399]])

In [9]:
# Recomendamos
suggested = []
for user in kusers[0]:
  for item in user_items[user]:
    if not item in suggested: suggested.append(item)

In [10]:
print(suggested)

[19, 17, 23, 2, 9, 11, 24, 12, 8, 14, 13, 4, 21, 20, 10, 3, 6, 5, 22, 15, 18]


## 2. Sistema basado en contenido

In [11]:
n_items = 500
items = np.zeros(shape=(n_items,4))

for i in range(n_items):
  items[i,0] = np.random.randint(0,5)
  items[i,1] = np.random.randint(0,3)
  items[i,2] = np.random.randint(0,10)
  items[i,3] = np.random.randn()

In [12]:
# Clustering
knn = NearestNeighbors(n_neighbors=10, radius=1.2)
knn.fit(items)

NearestNeighbors(n_neighbors=10, radius=1.2)

In [13]:
test_item = np.array([1,2,1,0.5]).reshape(1,-1)
dist, kitems = knn.kneighbors(test_item)

In [14]:
kitems

array([[370, 292,  28, 197, 420, 351, 230, 291, 276,   0]])

## 3. Sistema basado en memoria, filtro colaborativo

In [15]:
n_users=100
n_items=25

M = np.random.randint(-1,11, size=(n_users,n_items))

In [16]:
! pip install scikits.crab 

ERROR: Could not find a version that satisfies the requirement scikits.crab (from versions: none)
ERROR: No matching distribution found for scikits.crab


In [17]:
matrix_user_items = {
    1: {1:-1, 2:-1, 3:4},
    2: {1:5, 2:-1, 3:4},
    3: {1:3, 2:-2, 3:3},
    4: {1:2, 2:-6, 3:4},
    5: {1:-1, 2:6, 3:2}
}

In [18]:
knn = NearestNeighbors(n_neighbors=6, radius=1.0)

In [19]:
knn.fit(M)

NearestNeighbors(n_neighbors=6)

In [20]:
dist, kusers = knn.kneighbors(M[4].reshape(1,-1))

In [21]:
kusers

array([[ 4, 76, 79, 69, 81, 90]])

In [22]:
M[kusers[0]]

array([[ 5, 10,  7,  3,  9, 10, -1,  6,  4,  3,  8,  3,  9,  5, -1,  6,
         4,  3, 10, 10,  9,  4,  1,  1,  0],
       [ 8,  4,  0,  4, 10,  9, -1,  8,  1,  5,  7,  5,  6,  1,  7,  6,
         3,  9,  3,  5,  9,  3,  0,  2,  0],
       [-1,  7,  9, 10,  9,  6,  6,  6,  0,  4,  9,  6,  9,  7,  0,  8,
         1,  3,  2,  4,  7,  3, -1,  5,  1],
       [ 1,  6,  3,  9,  8,  8,  2,  8,  1,  5,  9,  2,  7,  9, -1, 10,
        -1,  8,  5,  5,  1,  8,  0,  5,  3],
       [ 8, 10, -1,  1, -1,  2,  0,  4,  6,  3,  2,  0,  8,  7,  2,  7,
         0,  7,  8,  7,  8,  7, -1,  2,  1],
       [ 8,  7,  6, -1,  7,  9,  3,  7, 10,  8,  8,  9,  5,  5, -1,  6,
         9,  5, 10,  2,  3,  1,  8,  6,  1]])

In [23]:
np.argmax(M[kusers[0]], axis=1)

array([ 1,  4,  3, 15,  1,  8])

In [24]:
! pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 28.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630172 sha256=f498bbc83fd830333c07412b5b1c40e0ccae2d773379f3b9dcc187b2417879a3
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


## 4. SVD

In [25]:
import surprise

In [30]:
from surprise import Dataset
dataset = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [33]:
# Algoritmo
from surprise import SVD
svd = SVD()

In [36]:
from surprise.model_selection import cross_validate
cross_validate(svd, dataset, measures=['RMSE'], cv=5)

{'fit_time': (5.308938980102539,
  5.154662132263184,
  4.98231840133667,
  4.950488567352295,
  4.978787183761597),
 'test_rmse': array([0.93701838, 0.93350633, 0.93936559, 0.93208236, 0.94136229]),
 'test_time': (0.15789246559143066,
  0.26044750213623047,
  0.15096116065979004,
  0.1567974090576172,
  0.3017768859863281)}

In [41]:
dataset.raw_ratings[:3]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116')]

In [43]:
users = [196, 186, 22]
users = [str(x) for x in users]
items = [242, 302, 377]
items = [str(x) for x in items]

In [50]:
svd.predict(users[2], items[2], r_ui=3.0, clip=True)

Prediction(uid='22', iid='377', r_ui=3.0, est=2.015353130587728, details={'was_impossible': False})

In [53]:
svd.predict(users[2], str(992), r_ui=3.0, clip=True)

Prediction(uid='22', iid='992', r_ui=3.0, est=3.224320996057176, details={'was_impossible': False})